In [ ]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib
%matplotlib inline
matplotlib.style.use('default')
import requests
import json

In [ ]:
# data and google api key setup
#google_api_access_key = open('../.env/.google_api_access_key').read()
google_api_access_key = open('../.env/.key').read()

In [ ]:
# let's use our great df, we created in Grid Generator notebook
df_grid_points_final = pd.read_excel('../raw_data/raw_data_geneva_grid_points.xlsx', usecols='B:D') 
df_grid_points_final

In [ ]:
# convert grid points to numpy array for an efficient iteration which is far more efficient than iterrows()
grid_points_array = df_grid_points_final[['lat','long']].to_numpy()
print(grid_points_array.shape)
grid_points_array

# Google Places Nearby Search

In [ ]:
# Sample coordinate
lat, lng = grid_points_array[1][0], grid_points_array[1][1]
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'keyword':'restaurant',
    'location':f'{lat},{lng}',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['name',
          'place_id',
          'types',
          'price_level',
          'rating',
          'user_ratings_total',
          'vicinity',
          'plus_code.global_code',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

In [ ]:
%%time
#this is my test to check the git (Tchucky)
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?' # base url

df_restaurants = pd.DataFrame()

for row in grid_points_array:
    lat, lng = row[0], row[1]
    radius = 100 #meters

    params = {
        'keyword':'restaurant',
        'location':f'{lat},{lng}',
        'radius':radius,
        'region':'ch',
        'key':google_api_access_key
    }
    
    try:
        response_basic = requests.get(base_url, params=params).json()

        # create a temporarily dataframe to hold response
        df_temp= pd.json_normalize(response_basic['results'])\
            [['name',
              'place_id',
              'types',
              'price_level',
              'rating',
              'user_ratings_total',
              'vicinity',
              'plus_code.global_code',
              'geometry.location.lat',
              'geometry.location.lng']]

        # concat and remove duplicates (beware drop_duplicates do not work due to lists values)
        drop_ref = df_temp.drop(columns=['types']).columns.to_list()
        df_restaurants = pd.concat([df_restaurants, df_temp], axis=0)\
            .drop_duplicates(drop_ref)\
            .reset_index(drop=True)
        
    except:
        continue
print(f'last_row: {row}')

In [ ]:
df_restaurants

In [ ]:
df_restaurants.to_csv('../raw_data/raw_data_google_api.csv', encoding='utf-8-sig')

# Google Places Details Search

In [ ]:
# Sample response / Google Places Details search
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'

place_id = 'ChIJX68uubZkjEcRKKxpZ7OrgqU'

# field dimension to investigate
params = {
    'place_id':place_id,
    'key':google_api_access_key
}

# filter also columns 
response_details_sample = requests.get(base_url, params=params).json()
df_details_sample = pd.json_normalize(response_details_sample)
df_details_sample

In [ ]:
response_details_sample